#Import


In [3]:
!pip3 install pyinflect
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [4]:
from nltk.stem import WordNetLemmatizer  
lemmatizer = WordNetLemmatizer() 
from pyinflect import getAllInflections, getInflection
from nltk import word_tokenize
import spacy
import re
import spacy
nlp = spacy.load("en_core_web_sm")
import csv
import json

In [5]:
!pip install autocorrect
from autocorrect import Speller
spell = Speller(lang='en')

In [6]:
!git clone https://github.com/nozomiyamada/contest2

fatal: destination path 'contest2' already exists and is not an empty directory.


#Function for generating candidate

In [7]:
#noun.csv
noun_dict2 = dict()
noun_list = []
with open('/content/contest2/noun.json') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  for row in csv_reader:
    for string in row:
      noun_list.append(string.strip(' ').strip("]").strip("[").strip(' " '))
for i in range(len(noun_list)-1):
  noun_dict2[noun_list[i]] = noun_list[i : i+2]

#verb.csv
verb_dict2 = dict()
with open('/content/contest2/verbs-dictionaries.csv') as csv_file:
  csv_reader = csv.reader(csv_file,delimiter =',')
  for row in csv_reader:
    for verbs in row:
      verbs_list = verbs.split('\t')
      verb_dict2[verbs_list[0]] = verbs_list[:]

#prep.json
prep_list = []
with open('/content/contest2/prep.json') as f:
  data = json.load(f)
  #prep_list.append("")
  for prep in data:
    prep_list.append(prep)

In [8]:
#find candidates

def pos_tag(sentence):
   tokens = word_tokenize(sentence)
   return nltk.pos_tag(tokens)

def lemma(word):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(word)
  lemma_word = [token.lemma_ for token in doc]
  return lemma_word

def verb_form_candidate(word):
  infection_dict = getAllInflections(word , 'V')
  if word in verb_dict2:
    get_infection = verb_dict2[word]
  else:
    get_infection = [tup[0] for tup in infection_dict.values()]
    get_infection.append(word)
  return get_infection

def be_verb():
  return ['be', 'is', 'am', 'are','was', 'were', 'been','being']

def article_candidate():
  return ['an','a','the',"some",'any','this','that','those','these', '']

def  noun_candidate(word):
  noun_dict1 = getAllInflections(word , 'N')
  if len(word) > 1 and word in noun_dict2:
    get_forms = noun_dict2[word]
  else:
    get_forms = [tup[0] for tup in noun_dict1.values()]
    if len(get_forms) == 0:
      get_forms.append(word)
      get_forms.append(word + 's')
      if word.endswith('y') and len(word) > 1 and word[-2] not in ['a','e','i','o','u']:
        get_forms.append(word[ : -1] + 'ies')
  return get_forms

def prep_candidate():
   return prep_list

#LM Model

In [9]:
#Install KenLM Library
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mkdir kenlm/build

--2020-08-18 12:58:25--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 490441 (479K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 478.95K   516KB/s    in 0.9s    

2020-08-18 12:58:27 (516 KB/s) - written to stdout [490441/490441]

mkdir: cannot create directory ‘kenlm/build’: File exists


In [10]:
%cd kenlm/build
!cmake ..
!make -j2

/content/kenlm/build
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   program_options
--   system
--   thread
--   unit_test_framework
--   chrono
--   date_time
--   atomic
-- Could NOT find Eigen3 (missing: EIGEN3_INCLUDE_DIR EIGEN3_VERSION_OK) (Required is at least version "2.91.0")
CMake Warning at lm/interpolate/CMakeLists.txt:65 (message):
  Not building interpolation.  Eigen3 was not found.


-- To install Eigen3 in your home directory, copy paste this:
export EIGEN3_ROOT=$HOME/eigen-eigen-07105f7124f9
(cd $HOME; wget -O - https://bitbucket.org/eigen/eigen/get/3.2.8.tar.bz2 |tar xj)
rm CMakeCache.txt

-- Configuring done
-- Generating done
-- Build files have been written to: /content/kenlm/build
[  5%] Built target kenlm_filter
[ 43%] Built target kenlm_util
[ 68%] Built target kenlm
[ 71%] Built target probing_hash_table_benchmark
[ 73%] Built target fragment
[ 76%] Built target build_binary
[ 78%] Built target kenlm_benchmark
[ 81%] Built target query
[ 9

In [11]:
#Dowload news data 
!gdown --id 1tGt5FJIZ0RPKAz4NxxhL1PvNd85AJBQu

Downloading...
From: https://drive.google.com/uc?id=1tGt5FJIZ0RPKAz4NxxhL1PvNd85AJBQu
To: /content/kenlm/build/gigaword3_nyt_eng.tar.gz
2.54GB [00:21, 116MB/s]


In [15]:
!tar -xf gigaword3_nyt_eng.tar.gz

In [13]:
#Dowload gutenberg
!gdown --id 1PeMJ8Z9HBDravgGzZ1baDRnliHRf1O75

Downloading...
From: https://drive.google.com/uc?id=1PeMJ8Z9HBDravgGzZ1baDRnliHRf1O75
To: /content/kenlm/build/Gutenberg.zip
462MB [00:02, 196MB/s]


In [16]:
from zipfile import ZipFile
with ZipFile('/content/kenlm/build/Gutenberg.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [17]:
ls #/content/kenlm/build/kenlm/build/kenlm

 bin/                                      nyt_eng_199801   nyt_eng_200209
 CMakeCache.txt                            nyt_eng_199802   nyt_eng_200210
 CMakeFiles/                               nyt_eng_199803   nyt_eng_200211
 cmake_install.cmake                       nyt_eng_199804   nyt_eng_200212
'decompose_n_add_token[i]_in_J.txt'        nyt_eng_199805   nyt_eng_200301
 file_for_evaluate.txt                     nyt_eng_199806   nyt_eng_200302
 gec-test-set.txt                          nyt_eng_199807   nyt_eng_200303
 gigaword3_nyt_eng.tar.gz                  nyt_eng_199808   nyt_eng_200304
 Gutenberg/                                nyt_eng_199809   nyt_eng_200305
 Gutenberg.zip                             nyt_eng_199810   nyt_eng_200306
 lib/                                      nyt_eng_199811   nyt_eng_200307
 lm/                                       nyt_eng_199812   nyt_eng_200308
 Makefile                                  nyt_eng_199901   nyt_eng_200309
'notdecompose_but_add_tok

In [18]:
!cat nyt_eng_2001* nyt_eng_2006*  > nyt_eng.txt

In [19]:
#Install KenLM in Python
!pip install https://github.com/kpu/kenlm/archive/master.zip

  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Created wheel for kenlm: filename=kenlm-0.0.0-cp36-cp36m-linux_x86_64.whl size=2330484 sha256=cc96248c35c79d2df703e31d84435d5206980a0d50fd4a6ce772b018e1506f17
  Stored in directory: /tmp/pip-ephem-wheel-cache-wai1fi81/wheels/2d/32/73/e3093c9d11dc8abf79c156a4db1a1c5631428059d4f9ff2cba
Successfully built kenlm


In [20]:
#Train LM with KenLM
!bin/lmplz -o 6 --text nyt_eng.txt --arpa sixgram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/nyt_eng.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1720688640 bytes == 0x557b808f6000 @  0x7fbfd2f791e7 0x557b7f9e8772 0x557b7f97c358 0x557b7f95b290 0x557b7f947096 0x7fbfd1112b97 0x557b7f948ada
tcmalloc: large alloc 9176997888 bytes == 0x557be71f0000 @  0x7fbfd2f791e7 0x557b7f9e8772 0x557b7f9d27aa 0x557b7f9d31c8 0x557b7f95b2ad 0x557b7f947096 0x7fbfd1112b97 0x557b7f948ada
****************************************************************************************************
Unigram tokens 209075440 types 880985
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:10571820 2:663265024 3:1243621888 4:1989795072 5:2901784576 6:3979590144
tcmalloc: large alloc 3979591680 bytes == 0x557b808f6000 @  0x7fbfd2f791e7 0x557b7f9e8772 0x557b7f9d27aa 0x557b7f9d31c8 0x557b7f95b84e 0x557b7f947096 0x7fbfd1112b97 0x557b7f948ada
tcmalloc: lar

In [21]:
ls

 bin/                                      nyt_eng_199801   nyt_eng_200209
 CMakeCache.txt                            nyt_eng_199802   nyt_eng_200210
 CMakeFiles/                               nyt_eng_199803   nyt_eng_200211
 cmake_install.cmake                       nyt_eng_199804   nyt_eng_200212
'decompose_n_add_token[i]_in_J.txt'        nyt_eng_199805   nyt_eng_200301
 file_for_evaluate.txt                     nyt_eng_199806   nyt_eng_200302
 gec-test-set.txt                          nyt_eng_199807   nyt_eng_200303
 gigaword3_nyt_eng.tar.gz                  nyt_eng_199808   nyt_eng_200304
 Gutenberg/                                nyt_eng_199809   nyt_eng_200305
 Gutenberg.zip                             nyt_eng_199810   nyt_eng_200306
 lib/                                      nyt_eng_199811   nyt_eng_200307
 lm/                                       nyt_eng_199812   nyt_eng_200308
 Makefile                                  nyt_eng_199901   nyt_eng_200309
'notdecompose_but_add_tok

In [35]:
import kenlm
model = kenlm.Model('sixgram.arpa')

#Evaluate Function

In [22]:
def evaluate(model, s):
  tokens = s.split(' ')
  log_score = 0.0
  for i, (logprob, length, oov) in enumerate(model.full_scores(s,bos = True, eos = True)):
    if i < len(tokens):
     tokens[i], math.exp(logprob), oov
    else:
      'END', math.exp(logprob), oov
  
    log_score += logprob
  return log_score

In [23]:
import math
def print_score(model, s):
  tokens = s.split(' ')
  log_score = 0.0
  for i, (logprob, length, oov) in enumerate(model.full_scores(s)):
    if i < len(tokens):
      print(tokens[i], math.exp(logprob), oov)
    else:
      print('END', math.exp(logprob), oov)
  
    log_score += logprob
  return log_score

#Edit function

In [24]:
def edit_noun(word,tokens,i):
      sentence_candidates = []
      default = word
      stem = lemmatizer.lemmatize(word.lower())
      for candidate in noun_candidate(stem):
          new_tokens = tokens[:]
          new_tokens[i] = candidate
          new_sentence = " ".join(new_tokens)
          sentence_candidates.append((candidate,evaluate(model,new_sentence)))
      rank = sorted(sentence_candidates , key = lambda x:x[1],reverse=True)
      best_candidate = rank[0][0]
      if default[0].isupper() == True:
        best_candidate = best_candidate.capitalize()
      return best_candidate

In [25]:
 def edit_art(word,tokens,i): 
      sentence_candidates = []
      for candidate in article_candidate():
          new_tokens = tokens[:]
          new_tokens[i] = candidate
          new_sentence = " ".join(new_tokens)
          sentence_candidates.append((candidate,evaluate(model,new_sentence)))
      rank = sorted(sentence_candidates , key = lambda x:x[1],reverse=True)
      best_candidate = rank[0][0]
      return best_candidate

In [26]:
def edit_be_verb(word,tokens,i):    
      sentence_candidates = []
      for candidate in be_verb():
          new_tokens = tokens[:]
          new_tokens[i] = candidate
          new_sentence = " ".join(new_tokens)
          sentence_candidates.append((candidate,evaluate(model,new_sentence)))
      rank = sorted(sentence_candidates , key = lambda x:x[1],reverse=True)
      best_candidate = rank[0][0]
      return best_candidate

In [27]:
def edit_verb(word,tokens,i):
      sentence_candidates = []
      default = word
      stem = nltk.stem.WordNetLemmatizer().lemmatize(word, 'v')

      for candidate in verb_form_candidate(stem):
        new_tokens = tokens[:]
        new_tokens[i] = candidate
        new_sentence = " ".join(new_tokens)
        sentence_candidates.append((candidate,evaluate(model,new_sentence)))
      rank = sorted(sentence_candidates , key = lambda x:x[1],reverse=True)
      best_candidate = rank[0][0]
      if default[0].isupper() == True:
        best_candidate = best_candidate.capitalize()
      return best_candidate

In [28]:
def edit_passive_voice(word,tokens,i):
      sentence_candidates = []
      stem = lemmatizer.lemmatize(word.lower())
      if stem in verb_form_candidate(stem):
        for vj_can in verb_form_candidate(stem):        
          new_tokens = tokens[:]
          new_tokens[i] = vj_can
          sentence = " ".join(new_tokens)
          sentence_candidates.append((vj_can,evaluate(model,sentence)))
        rank = sorted(sentence_candidates , key = lambda x:x[1],reverse=True)
        best_candidate = rank[0][0]
      else:
        best_candidate = word
      return best_candidate

In [29]:
def edit_prep(word,tokens,i):   
      sentence_candidates = []
      for candidate in prep_candidate():
        new_tokens = tokens[:]
        new_tokens[i] = candidate
        new_sentence = " ".join(new_tokens)
        sentence_candidates.append((candidate,evaluate(model,new_sentence)))
      rank = sorted(sentence_candidates , key = lambda x:x[1],reverse=True)
      best_candidate = rank[0][0]
      return best_candidate

In [30]:
def join_tokens(tokens):
  result = " ".join(tokens)
  if len(result) > 2:
    result = result[0].upper() + result[1:]
  else:
    result = result
  return result

In [31]:
def edit_sentence7(sentence):
  pos_sentence = pos_tag(sentence)
  urls = re.findall(r"https?", sentence)
  if len(urls) > 0:
    return sentence
  else:
   tokens = [token for token,pos in pos_sentence]
   for i in range(len(pos_sentence)): 
  
  #Noun    
    if pos_sentence[i][1].startswith("N"):
      tokens[i] = edit_noun(pos_sentence[i][0],tokens,i)

    #Article
    elif pos_sentence[i][0] in article_candidate():
      tokens[i] = edit_art(pos_sentence[i][0],tokens,i)

    #Be_verb
    elif pos_sentence[i][0] in be_verb():
      tokens[i] = edit_be_verb(pos_sentence[i][0],tokens,i)

    #Verb_forms
    elif pos_sentence[i][1].startswith("V"):
      tokens[i] = edit_verb(pos_sentence[i][0],tokens,i)

    elif pos_sentence[i][1].startswith("J"):
      tokens[i] = edit_passive_voice(pos_sentence[i][0],tokens,i)

    #Preposition
    elif pos_sentence[i][0] in prep_candidate():
      tokens[i] = edit_prep(pos_sentence[i][0],tokens,i)
  return join_tokens(tokens)

#Evluate

In [32]:
with open('/content/contest2/bea19-sentences.txt') as file:
  data = file.read().split('\n')
  data = [s for s in data]
data[100]

"According Khandani 's article , engineers can test the solution by prototyping or concurrent engineering ."

In [36]:
from tqdm import tqdm_notebook
with open('file_for_evaluate.txt', 'w') as f :
  for i in tqdm_notebook(range(len(data))):
    if i+1 < len(data):
      f.write(edit_sentence7(data[i]) + '\n')
    else:
      f.write(edit_sentence7(data[i]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
!python /content/contest2/m2scorer/scripts/m2scorer.py /content/kenlm/build/file_for_evaluate.txt /content/contest2/nucle.train.gold.bea19.m2

Precision   : 0.0680
Recall      : 0.1767
F_0.5       : 0.0775


#Generate Test file

In [38]:
!gdown --id 1CHnRNybDYbq9xTZNCxf22PIQIyBe0Yfz

Downloading...
From: https://drive.google.com/uc?id=1CHnRNybDYbq9xTZNCxf22PIQIyBe0Yfz
To: /content/kenlm/build/gec-test-set.txt
100% 162k/162k [00:00<00:00, 69.4MB/s]


In [39]:
with open('/content/kenlm/build/gec-test-set.txt') as file:
  data = file.read().split('\n')
  sentences = [s for s in data]
sentences[100]

"Genetic testing is made possible and available for one 's decision to undergo ."

In [41]:
from tqdm import tqdm_notebook
with open('corrected_sentences.txt', 'w') as f :
  for i in tqdm_notebook(range(len(sentences))):
    if i+1 < len(data):
      f.write(edit_sentence7(sentences[i]) + '\n')
    else:
      f.write(edit_sentence7(sentences[i]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until
